In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df_train_data = pd.read_excel("/content/drive/MyDrive/My Zoom Datasets/train.xlsx")
df_train_data

,text,reason,label
0,this is an amazing app for online classes!but,good app for conducting online classes,1
1,very practical and easy to use,app is user-friendly,1
2,this app is very good for video conferencing.,good for video conferencing,1
3,i can not download this zoom app,unable to download zoom app,1
4,i am not able to download this app,want to download the app,1
...,...,...,...
2056,i am not getting jio tv properly.,unable to use with jio tv,1
2057,(learn from the netflix interface),netflix ui is better,1
2058,"hello,\ndisney+ must also be installed on chro...",unable to use disney on chromecast,1
2059,it is a shame that disney+ does not work on tv...,unable to use disney plus on tv,1


In [11]:
df_train_data["label"].value_counts()

,count
label,
1,2061


In [15]:
df_evaluation_data = pd.read_excel("/content/drive/MyDrive/My Zoom Datasets/evaluation.xlsx")
df_evaluation_data

,text,reason,label
0,the app is crashing when i play a vedio,app crashes during playback,1
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0
2,very helpful when and home working remotley,good app for work,0
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0
4,one of my favorite apps,good for spending time,0
...,...,...,...
8995,very good streaming platform has a lot of content,app has very good content,1
8996,so paying for an app that is not fit for purpose.,unable to use app,0
8997,"it does not work well for me, it loads, but if...",app keeps loading indefinitely,0
8998,i can not watch the new disney shows.,unable to load the episodes list,0


In [16]:
df_evaluation_data["label"].value_counts()

,count
label,
0,5999
1,3001


In [2]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 102.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [4]:
pip install --upgrade transformers datasets

In [17]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset
import gradio as gr
import os
os.environ["WANDB_DISABLED"] = "true"  # ← disables W&B

# Load data
train_df = pd.read_excel("/content/drive/MyDrive/My Zoom Datasets/train.xlsx")
eval_df = pd.read_excel("/content/drive/MyDrive/My Zoom Datasets/evaluation.xlsx")

# Combine text and reason as sentence pairs
train_df['text_pair'] = list(zip(train_df['text'], train_df['reason']))
eval_df['text_pair'] = list(zip(eval_df['text'], eval_df['reason']))

#(train_df)
# Only shows:
# label 1 → 2061
# 🔴 Missing label 0 entirely.
# This means it's not suitable for training, as the model won't learn to recognize class 0.


# (elaluation dataset)
# It contains both class 0 and class 1.
# The class distribution is:
# Class 0: 5999
# Class 1: 3001
# Which is acceptable with slight class imbalance that you can handle using class weights.


# Train-test split
train_data, val_data = train_test_split(eval_df, test_size=0.1, stratify=eval_df['label'], random_state=42)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['text'], example['reason'], padding="max_length", truncation=True)

# Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data[['text', 'reason', 'label']])
val_dataset = Dataset.from_pandas(val_data[['text', 'reason', 'label']])
eval_dataset = Dataset.from_pandas(eval_df[['text', 'reason', 'label']])

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",           # For evaluation
    save_strategy="epoch",           # For saving models
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_total_limit=1,
    logging_dir="./logs",
    load_best_model_at_end=True,
)


# Define compute_metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return classification_report(p.label_ids, preds, output_dict=True)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Evaluate
print("Evaluation Report:")
preds_output = trainer.predict(eval_dataset)
print(classification_report(preds_output.label_ids, preds_output.predictions.argmax(-1)))

# Save model and tokenizer
model.save_pretrained("./myzoom_model")
tokenizer.save_pretrained("./myzoom_model")


Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.434200,0.373246,"{'precision': 0.9144851657940664, 'recall': 0.8733333333333333, 'f1-score': 0.8934356351236147, 'support': 600.0}","{'precision': 0.7675840978593272, 'recall': 0.8366666666666667, 'f1-score': 0.8006379585326954, 'support': 300.0}",0.861111,"{'precision': 0.8410346318266968, 'recall': 0.855, 'f1-score': 0.847036796828155, 'support': 900.0}","{'precision': 0.8655181431491533, 'recall': 0.8611111111111112, 'f1-score': 0.8625030762599749, 'support': 900.0}"
2,0.304500,0.354560,"{'precision': 0.9206611570247933, 'recall': 0.9283333333333333, 'f1-score': 0.9244813278008299, 'support': 600.0}","{'precision': 0.8542372881355932, 'recall': 0.84, 'f1-score': 0.8470588235294118, 'support': 300.0}",0.898889,"{'precision': 0.8874492225801933, 'recall': 0.8841666666666667, 'f1-score': 0.8857700756651208, 'support': 900.0}","{'precision': 0.89851986739506, 'recall': 0.8988888888888888, 'f1-score': 0.8986738263770239, 'support': 900.0}"
3,0.163400,0.422759,"{'precision': 0.9402730375426621, 'recall': 0.9183333333333333, 'f1-score': 0.9291736930860034, 'support': 600.0}","{'precision': 0.8439490445859873, 'recall': 0.8833333333333333, 'f1-score': 0.8631921824104235, 'support': 300.0}",0.906667,"{'precision': 0.8921110410643247, 'recall': 0.9008333333333334, 'f1-score': 0.8961829377482134, 'support': 900.0}","{'precision': 0.9081650398904372, 'recall': 0.9066666666666666, 'f1-score': 0.9071798561941434, 'support': 900.0}"


Trainer is attempting to log a value of "{'precision': 0.9144851657940664, 'recall': 0.8733333333333333, 'f1-score': 0.8934356351236147, 'support': 600.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7675840978593272, 'recall': 0.8366666666666667, 'f1-score': 0.8006379585326954, 'support': 300.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8410346318266968, 'recall': 0.855, 'f1-score': 0.847036796828155, 'support': 900.0}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8655181431491533, 'recall': 0.8611

Evaluation Report:


              precision    recall  f1-score   support

           0       0.96      0.97      0.96      5999
           1       0.94      0.92      0.93      3001

    accuracy                           0.95      9000
   macro avg       0.95      0.94      0.95      9000
weighted avg       0.95      0.95      0.95      9000



('./myzoom_model/tokenizer_config.json',
 './myzoom_model/special_tokens_map.json',
 './myzoom_model/vocab.txt',
 './myzoom_model/added_tokens.json')

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Gradio app
def classify_feedback(text, reason):
    try:
        inputs = tokenizer(text, reason, return_tensors="pt", truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        return "Aligned (1)" if prediction == 1 else "Not Aligned (0)"
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
gr.Interface(
    fn=classify_feedback,
    inputs=[
        gr.Textbox(label="Feedback Text"),
        gr.Dropdown(choices=train_df['reason'].unique().tolist(), label="Select Reason")
    ],
    outputs=gr.Textbox(label="Output"),
    title="Feedback Validation (My Zoom)"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73ecfceec1aa3c5ce3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
